In [1]:
import pandas as pd

def read_tsv( path ):
    """
    Read a tab-separated file with a header row.
    """
    return pd.read_csv( path, sep = "\t" )

train = read_tsv( "reminder_train.tsv" )
val   = read_tsv( "reminder_eval.tsv" )
test  = read_tsv( "reminder_test.tsv" )

train.sample( frac = 1 ).head( 10 )

,domain,utterance,semantic_parse
10667,reminder,On Saturday change my reminder alert from 4 pm...,[IN:UPDATE_REMINDER_DATE_TIME [SL:DATE_TIME On...
7191,reminder,edit alert to reminder to pay credit card from...,[IN:UPDATE_REMINDER_DATE_TIME edit alert to re...
9936,reminder,can you find me reminders of meeting times thi...,[IN:GET_REMINDER can you find me reminders of ...
3189,reminder,How many reminders to I have today?,[IN:GET_REMINDER How many reminders to [SL:PER...
14606,reminder,Cancel my reminders.,[IN:DELETE_REMINDER Cancel [SL:PERSON_REMINDED...
14727,reminder,remind me to check the app at 6pm today,[IN:CREATE_REMINDER remind [SL:PERSON_REMINDED...
6746,reminder,Move the reminder to pack for the camping trip...,[IN:UPDATE_REMINDER_DATE_TIME Move the reminde...
2020,reminder,I'd like you to remind me that my mother's fli...,[IN:CREATE_REMINDER I'd like you to remind [SL...
3736,reminder,Can you remind me an hour before I have to lea...,[IN:CREATE_REMINDER Can you remind [SL:PERSON_...
14183,reminder,Set a reminder for my doctor's appointment tom...,[IN:CREATE_REMINDER Set a reminder for [SL:PER...


In [2]:
#concatenate all data
combined = pd.concat( [ train, val, test ] )

#extract intent
def extract_intent( row ):
    """Extracts intent from input format.
    i.e. "IN:GET_REMINDER Can you find me reminders of the event" -> "GET_REMINDER"
    """
    return row['semantic_parse'][row['semantic_parse'].find('IN:')+3:row['semantic_parse'].find(' ')]

def correct_intent( row ):
    #Turn 'GET_REMINDER' into 'Get Reminder.'
    return row['intent'].replace( '_', ' ' ).title() + '.'

#Fix Intent Labels
combined['intent'] = combined.apply( extract_intent, axis = 1 )
combined['intent'] = combined.apply( correct_intent, axis = 1 )

#Drop Semantic Parse Column and 'Domain' Column
combined = combined.drop( columns = ['semantic_parse', 'domain'] )
combined.head( 10 )

,utterance,intent
0,remind me to write thank you letters to invited,Create Reminder.
1,Remind me of the information in the note secti...,Create Reminder.
2,remind me to take my meds at 8am and 6pm daily,Create Reminder.
3,I need to text Nicquana tonight at 7pm. Can yo...,Create Reminder.
4,Remind me to change my flight to New York.,Create Reminder.
5,delete my reminder to do my homework tonight,Delete Reminder.
6,please remind me on Saturday at 9 am to leave ...,Create Reminder.
7,Remind me 30 min before the Fluid dynamics exa...,Create Reminder.
8,Remind me at 5PM to schedule my hair appointme...,Create Reminder.
9,REMIND MY BAND TO BRING THEIR GEAR FOR TOMORRO...,Create Reminder.


In [3]:
combined.to_csv("data.csv")